Author: Lani Cupo       Date: 20190930
This script uses a "pilot" to simulate data for a power analysis using the simr package in R. 
Tutorials used include: 
https://rstudio-pubs-static.s3.amazonaws.com/484106_6b51212f20164fdd88cd7cce89bdef79.html
https://cran.r-project.org/web/packages/simr/vignettes/examples.html


In [1]:
#load all necessary libraries
#On CIC computers, simr is installed in R 3.5.1
library(lme4)
library(simr)
library(lmerTest)


Loading required package: Matrix


ERROR: Error in library(simr): there is no package called ‘simr’


In [2]:
#set working directory
setwd("/home/lani/phd1/pwr_analysis")

For this simulation:
my outcome (y) <- brain volume (of specific region)
group level predictors (fixed effects) <- treatment + sex + age
individual level predictors (random effects) <- mouse_id

In [3]:
#load in data sets
volumes <- read.csv("allvolumes_mia.csv")


In [4]:
#Test-retest betas for limit
sal_only <- subset(volumes, volumes$new_groups == "SAL")
#Drop levels
sal_only$new_groups <- factor(sal_only$new_groups)

sal_adults <- subset(sal_only, sal_only$timepoint %in% c(2,3))

retest_mod <- lmer(left_CA1Rad ~ sex + age + (1|mouse_id), data = sal_adults, REML = FALSE)
summary(retest_mod)
#This becomes the effect size for our model to test the limits
fixef(retest_mod)["age"]

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: left_CA1Rad ~ sex + age + (1 | mouse_id)
   Data: sal_adults

     AIC      BIC   logLik deviance df.resid 
  -150.1   -138.1     80.1   -160.1       77 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-2.03608 -0.69030  0.01228  0.64502  2.03518 

Random effects:
 Groups   Name        Variance Std.Dev.
 mouse_id (Intercept) 0.001923 0.04385 
 Residual             0.006604 0.08127 
Number of obs: 82, groups:  mouse_id, 41

Fixed effects:
             Estimate Std. Error t value
(Intercept) 1.5287049  0.0459823   33.25
sexmale     0.0535899  0.0225843    2.37
age         0.0008659  0.0005726    1.51

Correlation of Fixed Effects:
        (Intr) sexmal
sexmale -0.247       
age     -0.936 -0.005

age 
0.0008659141

In [5]:
# filter volumes to match my experimental design
#look at adolescent timepoints (1 and 2)
vols_adol <- subset(volumes, volumes$timepoint %in% c(1,2))
# Include only saline and poly early groups
vols_adol_early <- subset(vols_adol, vols_adol$new_groups %in% c("SAL", "POL_E"))
#Drop the extra factors
vols_adol_early$new_groups <- factor(vols_adol_early$new_groups)
#Exclude those missing a scan
#make a data frame representing how many scans a mouse has
n_occur <- data.frame(table(vols_adol_early$mouse_id))
#Subset the data frame to include those with 2 scans, only
two_scans <- subset(n_occur, n_occur$Freq %in% c(2))
#Drop extra factors for the mouse_id of mice 
two_scans$Var1 <- factor(two_scans$Var1)
vols_adol_early_noNA <- subset(vols_adol_early, vols_adol_early$mouse_id %in% two_scans$Var1)

In [6]:
# then get stats from lmer 
# Here I started with a specific regions
fm1 <- lmer(left_CA1Rad ~ sex + new_groups * age + (1|mouse_id), data = vols_adol_early_noNA, REML = FALSE)
summary(fm1)

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: left_CA1Rad ~ sex + new_groups * age + (1 | mouse_id)
   Data: vols_adol_early_noNA

     AIC      BIC   logLik deviance df.resid 
  -244.1   -225.1    129.1   -258.1      105 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.9358 -0.5020 -0.1015  0.5190  2.1120 

Random effects:
 Groups   Name        Variance Std.Dev.
 mouse_id (Intercept) 0.002831 0.05321 
 Residual             0.003661 0.06051 
Number of obs: 112, groups:  mouse_id, 56

Fixed effects:
                   Estimate Std. Error t value
(Intercept)        1.299369   0.048803  26.625
sexmale            0.048543   0.018302   2.652
new_groupsSAL      0.115432   0.057734   1.999
age                0.006581   0.000915   7.193
new_groupsSAL:age -0.003799   0.001102  -3.447

Correlation of Fixed Effects:
            (Intr) sexmal nw_SAL age   
sexmale     -0.159                     
new_grpsSAL -0.819 -0.031              
age         -0.929 -0.009 

In [7]:
#Create power curves-------------------------------------------------------------------
#https://rstudio-pubs-static.s3.amazonaws.com/484106_6b51212f20164fdd88cd7cce89bdef79.html
fixef(fm1)["new_groupsSAL"]

# variable identified is expaneded to n rows 
model_ext_mice <- extend(fm1, along="mouse_id", n=200)
model_ext_mice

new_groupsSAL 
    0.1154315

ERROR: Error in extend(fm1, along = "mouse_id", n = 200): could not find function "extend"


In [ ]:
#Set effect size for the actual model to analyze what our power is like now.
fixef(fm1)["new_groupsSAL"] <- 0.15
#fixef(fm1)["new_groupsSAL"] <- 0.3
#Analyze what power we have with the given sample size
powerSim(fm1, test = fixed("new_groupsSAL", "t"))
#Set effect size for the extended model 
fixef(model_ext_mice)["new_groupsSAL"] <- 0.15
#fixef(model_ext_mice)["new_groupsSAL"] <- 0.3
set.seed(0)
p_curve_treat <- powerCurve(model_ext_mice, 
                          test= fixed("new_groupsSAL", "t"), 
                          along="mouse_id",
                          breaks=seq(1,100,10),
                          nsim=100,alpha=.05, progress=TRUE)
plot(p_curve_treat)

In [ ]:
#Test smallest possible effect size (from test-retest)
#First use real data
fixef(fm1)["new_groupsSAL"] <- fixef(retest_mod)["age"]
powerSim(fm1, test = fixed("new_groupsSAL", "t"))

#Then extend to larger sample size
fixef(model_ext_mice)["new_groupsSAL"] <- fixef(retest_mod)["age"]
powerSim(model_ext_mice, test = fixed("new_groupsSAL", "t"))
set.seed(0)
p_curve_bound <- powerCurve(model_ext_mice, 
                                test=fixed("new_groupsSAL", "t"), 
                                along="mouse_id",
                                breaks=seq(1,120, 20),
                                nsim=500,alpha=.05, progress=TRUE)
plot(p_curve_bound)


In the following section, I generate the histograms to examine our newly simulated data. 

In [ ]:
#Create variable for the new data
newData <- getData(model_ext_mice)
#Compare tables of the old data and the new data 
table(vols_adol_early_noNA$timepoint, vols_adol_early_noNA$new_groups)
table(vols_adol_early_noNA$new_groups, vols_adol_early_noNA$sex)

#Check the new data for the distributions -- are the proportions the same?
table(newData$timepoint, newData$new_groups)
table(newData$new_groups, newData$sex)

#histogram of structure (left CA1Rad) we are examining
hist_orig <- hist(vols_adol_early_noNA$left_CA1Rad,
                  main="Volume of left CA1 Rad",
                  xlab="Volumes (mm)")

#histogram of structure we are examining
hist_orig <- hist(newData$left_CA1Rad,
                  main="Volume of left CA1 Rad (simulated)",
                  xlab="Volumes (mm) (simulated)")
